In [41]:
import json

from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from tqdm.notebook import tqdm

In [39]:
def plot_icon_on_map(map_image: Image.Image, x: float, y: float, icon: Image.Image):
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    icon_size = 80

    icon = icon.resize((icon_size, icon_size), Image.LANCZOS)
    icon_width, icon_height = icon.size

    icon_x = coord_image_x - icon_width // 2
    icon_y = coord_image_y - icon_height // 2

    map_image.paste(icon, (icon_x, icon_y), icon)

    return map_image

def plot_circle_on_map(map_image: Image.Image, x: float, y: float, radius: float, color: tuple):
    # draw a filled circle on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    draw = ImageDraw.Draw(map_image)

    circle_x0 = coord_image_x - radius
    circle_y0 = coord_image_y - radius
    circle_x1 = coord_image_x + radius
    circle_y1 = coord_image_y + radius

    draw.ellipse([circle_x0, circle_y0, circle_x1, circle_y1], fill=color)

    return map_image

def plot_text_on_map(map_image: Image.Image, x: float, y: float, text: str, font: ImageFont.FreeTypeFont, color: tuple, angle: float = 0):
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    text_bounding_box = font.getbbox(text)
    text_width = text_bounding_box[2] - text_bounding_box[0]
    text_height = text_bounding_box[3] - text_bounding_box[1]
    text_image = Image.new('RGBA', (text_width, text_height))
    text_draw = ImageDraw.Draw(text_image)
    text_draw.text((0, 0), text, font=font, fill=color, anchor='lt')
    text_image = text_image.rotate(angle, expand=1)

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    text_x = coord_image_x - text_image.size[0] // 2
    text_y = coord_image_y - text_image.size[1] // 2 

    map_image.paste(text_image, (text_x, text_y), text_image)

    return map_image


def plot_box_on_map(map_image: Image.Image, start: tuple, end: tuple, color: tuple):
    # draw a filled box on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    start_image_x = int((start[0] - min_x) / (max_x - min_x) * image_width)
    start_image_y = int((start[1] - min_y) / (max_y - min_y) * image_height)

    end_image_x = int((end[0] - min_x) / (max_x - min_x) * image_width)
    end_image_y = int((end[1] - min_y) / (max_y - min_y) * image_height)

    # box_image = Image.new('RGBA', map_image.size, (0, 0, 0, 0))
    box_image = Image.new('RGBA', (end_image_x - start_image_x, end_image_y - start_image_y), (0, 0, 0, 0))
    draw = ImageDraw.Draw(box_image)

    # draw.rectangle([start_image_x, start_image_y, end_image_x, end_image_y], fill=color)
    draw.rectangle([0, 0, end_image_x - start_image_x, end_image_y - start_image_y], fill=color)
    
    map_image.paste(box_image, (start_image_x, start_image_y), box_image)

    return map_image

def plot_rounded_box_on_map(map_image: Image.Image, start: tuple, end: tuple, color: tuple, radius: int = 12):
    # draw a filled box on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    start_image_x = int((start[0] - min_x) / (max_x - min_x) * image_width)
    start_image_y = int((start[1] - min_y) / (max_y - min_y) * image_height)

    end_image_x = int((end[0] - min_x) / (max_x - min_x) * image_width)
    end_image_y = int((end[1] - min_y) / (max_y - min_y) * image_height)

    # box_image = Image.new('RGBA', map_image.size, (0, 0, 0, 0))
    box_image = Image.new('RGBA', (end_image_x - start_image_x, end_image_y - start_image_y), (0, 0, 0, 0))
    draw = ImageDraw.Draw(box_image)

    # draw.rectangle([start_image_x, start_image_y, end_image_x, end_image_y], fill=color)
    draw.rounded_rectangle([0, 0, end_image_x - start_image_x, end_image_y - start_image_y], fill=color, radius=radius)
    
    map_image.paste(box_image, (start_image_x, start_image_y), box_image)

    return map_image

def recolor_image(image: Image.Image, color: tuple):
    data = image.getdata()

    new_data = []
    for item in data:
        if item[0] >= 200 and item[1] >= 200 and item[2] >= 200:
            new_data.append((color[0], color[1], color[2], item[3]))
        else:
            new_data.append(item)

    image.putdata(new_data)

    return image

def rotate_image(image: Image.Image, angle: float):
    return image.rotate(angle, resample=Image.BICUBIC, expand=True)

In [17]:
world_path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.1.4512/json_data/Maine/Content/Levels/AR_00_World/AR_12_Sandbox/')
world_files = list(world_path.glob('*.json'))
world_json = []
for file in world_files:
    world_json.extend(json.loads(file.read_text()))

In [22]:
world_entries: dict[str, list] = {}
for entry in world_json:
    if 'Outer' not in entry:
        continue

    if entry['Outer'] not in world_entries:
        world_entries[entry['Outer']] = []
    world_entries[entry['Outer']].append(entry)

In [32]:
with open('world_entries.json', 'w') as f:
    string = json.dumps(list(world_entries.items())[0], indent=2)
    f.write(string)

In [48]:
map_image = Image.open(Path('D:/Backup/samjv/Desktop/code/python/grounded/combined_resized_image.png')).convert('RGBA')
map_image = map_image.resize((10000, 10000), Image.NEAREST)

for world_name, world in tqdm(world_entries.items()):
    print(world_name)
    for entry in world:
        if 'Properties' not in entry or 'RelativeLocation' not in entry['Properties']:
            continue

        coords = entry['Properties']['RelativeLocation']
        coords = (coords['X'], coords['Y'], coords['Z'])

        map_image = plot_circle_on_map(map_image, coords[0], coords[2], 10, (255, 0, 0, 255))
        map_image = plot_text_on_map(map_image, coords[0], coords[2], entry['Outer'], ImageFont.truetype('../GillSansMTPro-Condensed.ttf', 24), (255, 255, 255, 255))

    break

map_image.save('map_image.png')

  0%|          | 0/381 [00:00<?, ?it/s]

BP_Lab_Experiment_Chamber_A_Science_2
